In [ ]:
#Importing required libraries

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
#Reading .csv files

anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

In [ ]:
anime_df.head()

In [ ]:
rating_df.head()

In [ ]:
df = pd.merge(rating_df,anime_df.drop('rating',axis=1),on='anime_id')

In [ ]:
df.head()

In [ ]:
print(df.nunique(axis=0))
sns.barplot(x = df.columns, y = df.nunique(axis=0))

In [ ]:
#Spliting data into train and test data set

train, test = train_test_split(df, test_size=0.2, random_state=42)

train.head()

In [ ]:
test.head()

In [ ]:
finalRating = pd.DataFrame(df.groupby('name')['rating'].mean())
finalRating['totalRating'] = pd.DataFrame(df.groupby('name')['rating'].count())
finalRating.head()

In [ ]:
#Plotting the rating and total rating from the dfRating

sns.jointplot(x='totalRating',y='rating',data=finalRating,alpha=1)

In [ ]:
#Creating the pivot table for reorganizing the data and summarizing the rating of anime for each user.

animePivot = df.pivot_table(index='name',columns='user_id',values='rating').fillna(0)


In [ ]:
animePivot

In [ ]:
#Defining the KNN model with sparse matrix and brute algorithm.

sparseAnime=csr_matrix(animePivot)
model=NearestNeighbors(algorithm='brute')
model.fit(sparseAnime)

In [ ]:
#Defining the recommendation system method and course name as parameter.

def animeRecommendation(name):
    try:
        user_id=np.where(animePivot.index.str.lower()==name.lower())[0][0]
        distances, suggestions=model.kneighbors(animePivot.iloc[user_id,:].values.reshape(1,-1), n_neighbors=5)
        
        for i in range(len(suggestions)):
            print("Some recommendations for", name,"are:")
            print(animePivot.index[suggestions[i]])
    except:
        print("Sorry! couldn't find recommendation for the anime:",name)

In [ ]:
finalRating.sort_values('totalRating',ascending=False).head(30)

In [ ]:
#Taking user input for anime recommendation.

chooseAnime = input("Enter one anime: ")
print ('Selected anime:', chooseAnime)

In [ ]:
#Calling the recommendation method with user input

animeRecommendation(chooseAnime)